## Início do Pipeline de Modelagem e Rastreamento com MLflow

Este notebook tem como objetivo iniciar o pipeline de experimentos, carregando os datasets finais da camada `Curated` para criar os conjuntos de treino e teste totalmente alinhados. O fluxo inclui a separação de variáveis preditoras (`X`) e variável alvo (`y`), além da configuração inicial do MLflow Tracking, garantindo que todos os parâmetros, métricas e artefatos do modelo sejam rastreados de forma coerente e versionável.


In [1]:
#  ETAPA: Carga dos Dados Curados e Configuração do MLflow Tracking

"""
Executa:
1) Validação do diretório de trabalho.
2) Carregamento de 'train_curated.csv' e 'test_curated.csv'.
3) Separação de X_train, y_train, X_test, y_test.
4) Configuração do Tracking URI do MLflow.
"""

import os
import pandas as pd
import mlflow

# 1️⃣ Validar CWD
print("Current Working Directory:", os.getcwd())

# 2️⃣ Paths coerentes
TRAIN_PATH = 'data/curated/train_curated.csv'
TEST_PATH = 'data/curated/test_curated.csv'

# 3️⃣ Carregar datasets
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

print("\nTreino shape:", train_df.shape)
print("Teste shape:", test_df.shape)

# 4️⃣ Separar X e y
TARGET = 'Credit_Score_Standard'  # Ajuste para seu target real

X_train = train_df.drop(columns=[TARGET])
y_train = train_df[TARGET]

X_test = test_df.drop(columns=[TARGET])
y_test = test_df[TARGET]

print("\nX_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_test:", X_test.shape)
print("y_test:", y_test.shape)

# 5️⃣ Configurar MLflow Tracking URI
mlflow.set_tracking_uri("http://mlflow:5000")  # Ajuste se necessário
print("\nTracking URI configurado:", mlflow.get_tracking_uri())


Current Working Directory: /workspace

Treino shape: (100000, 6305)
Teste shape: (50000, 6305)

X_train: (100000, 6304)
y_train: (100000,)
X_test: (50000, 6304)
y_test: (50000,)

Tracking URI configurado: http://mlflow:5000


## Experimento Baseline com MLflow e Monitoramento de Progresso

Nesta etapa será rodado o primeiro experimento baseline usando o MLflow para rastrear parâmetros, métricas e artefatos. Para acompanhar operações potencialmente demoradas, como o ajuste do modelo (`fit`) e a geração de métricas, será utilizado o `tqdm` para monitorar loops de forma explícita. Isso garante visibilidade do progresso em tempo real, além de manter a rastreabilidade completa do pipeline.


In [2]:
#  ETAPA: Refazer Baseline — Normalização do Working Directory, Carga Curated, Reconstrução de y e Tracking MLflow

"""
Executa:
1) Validação e normalização do diretório de trabalho (CWD).
2) Carregamento de 'train_curated.csv' e 'test_curated.csv'.
3) Reconstrução de y a partir de colunas dummy.
4) Separação coerente de X e y.
5) Tracking URI e credenciais MinIO explícitos.
6) Treino DecisionTreeClassifier com barra de progresso.
7) Logging MLflow de hiperparâmetros, métricas e artefato.
8) Prints finais coerentes com links 127.0.0.1.
"""

import os
import logging
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm

# ✅ Silenciar logger redundante do MLflow
logging.getLogger("mlflow").setLevel(logging.ERROR)

# 1️⃣ Validar e corrigir CWD
print("Current Working Directory (antes):", os.getcwd())
os.chdir('/workspace')
print("Current Working Directory (depois):", os.getcwd())

# 2️⃣ Paths absolutos coerentes
TRAIN_PATH = 'data/curated/train_curated.csv'
TEST_PATH = 'data/curated/test_curated.csv'

# 3️⃣ Carregar datasets
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

print("\nTreino shape:", train_df.shape)
print("Teste shape:", test_df.shape)
print("\ntrain_df.head(5):")
print(train_df.head(5))

# 4️⃣ Reconstruir y a partir de colunas dummy
dummy_cols = [col for col in train_df.columns if col.startswith('Credit_Score_')]
print("\nColunas de classe detectadas:", dummy_cols)

# Reconstrói y_train
y_train = train_df[dummy_cols].idxmax(axis=1).str.replace('Credit_Score_', '')
X_train = train_df.drop(columns=dummy_cols)

# Reconstrói y_test
y_test = test_df[dummy_cols].idxmax(axis=1).str.replace('Credit_Score_', '')
X_test = test_df.drop(columns=dummy_cols)

print(f"\nX_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

# 5️⃣ Tracking URI e credenciais MinIO
mlflow.set_tracking_uri("http://mlflow:5000")
os.environ['AWS_ACCESS_KEY_ID'] = 'wrm'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'senha_segura'
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://minio:9000'

print("\nTracking URI:", mlflow.get_tracking_uri())
print("MLFLOW_S3_ENDPOINT_URL:", os.environ['MLFLOW_S3_ENDPOINT_URL'])

# 6️⃣ Cria/recupera experimento
experiment_name = "QuantumFinance_CreditScore"
mlflow.set_experiment(experiment_name)

with mlflow.start_run(run_name="Baseline_DecisionTree_Curated") as run:
    params = {"max_depth": 5, "random_state": 42}
    mlflow.log_params(params)

    model = DecisionTreeClassifier(**params)

    print("\nTreinando modelo com barra de progresso:")
    for _ in tqdm(range(1), desc="Fitting model"):
        model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    print(f"\nAccuracy: {acc:.4f}")
    print(f"F1 Score: {f1:.4f}")

    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("f1_score", f1)
    mlflow.sklearn.log_model(model, "model")

    # ✅ Prints finais coerentes — SOMENTE com 127.0.0.1
    print(f"\nRun ID: {run.info.run_id}")
    print(f"Acesse: http://127.0.0.1:5000/#/experiments/{run.info.experiment_id}/runs/{run.info.run_id}")
    print(f"View run Baseline_DecisionTree_Curated at: http://127.0.0.1:5000/#/experiments/{run.info.experiment_id}/runs/{run.info.run_id}")
    print(f"View experiment at: http://127.0.0.1:5000/#/experiments/{run.info.experiment_id}")


Current Working Directory (antes): /workspace/notebooks
Current Working Directory (depois): /workspace

Treino shape: (100000, 6305)
Teste shape: (50000, 6305)

train_df.head(5):
     Age  Annual_Income  Monthly_Inhand_Salary  Num_Bank_Accounts  \
0   23.0       19114.12            1824.843333                  3   
1   23.0       19114.12                    NaN                  3   
2 -500.0       19114.12                    NaN                  3   
3   23.0       19114.12                    NaN                  3   
4   23.0       19114.12            1824.843333                  3   

   Num_Credit_Card  Interest_Rate  Delay_from_due_date  Num_Credit_Inquiries  \
0                4              3                    3                   4.0   
1                4              3                   -1                   4.0   
2                4              3                    3                   4.0   
3                4              3                    5                   4.0   
4     

Fitting model: 100%|██████████| 1/1 [00:12<00:00, 12.90s/it]



Accuracy: 0.5496
F1 Score: 0.7093

Run ID: 86de72fd063e485fa584c1d8c0395aca
Acesse: http://127.0.0.1:5000/#/experiments/1/runs/86de72fd063e485fa584c1d8c0395aca
View run Baseline_DecisionTree_Curated at: http://127.0.0.1:5000/#/experiments/1/runs/86de72fd063e485fa584c1d8c0395aca
View experiment at: http://127.0.0.1:5000/#/experiments/1
🏃 View run Baseline_DecisionTree_Curated at: http://mlflow:5000/#/experiments/1/runs/86de72fd063e485fa584c1d8c0395aca
🧪 View experiment at: http://mlflow:5000/#/experiments/1


## ETAPA: Melhoria do Modelo com GridSearchCV e MLflow Tracking

Este bloco marca a transição do experimento baseline para uma etapa de otimização incremental, usando GridSearchCV para explorar múltiplas combinações de hiperparâmetros de forma sistemática e rastreada.

## Objetivo
- Encontrar a configuração de hiperparâmetros mais eficaz para a Árvore de Decisão (DecisionTreeClassifier).
- Registrar cada combinação testada como um run único no MLflow, com seus parâmetros, métricas e artefato final.

## Princípios aplicados
- Tracking URI e backend MinIO/S3 mantidos consistentes.
- Cada variação é rastreável, sem sobrescrever runs anteriores.
- Uso de tqdm para barra de progresso, garantindo visibilidade em loops demorados.
- Prints finais coerentes, com links 127.0.0.1 para acesso ao MLflow UI.

## Resultado esperado
- Métricas comparáveis entre baseline e grid search.
- Melhor modelo salvo como artefato no bucket MinIO.
- Próximo passo: preparar o pipeline para registrar o modelo validado no Registry do MLflow.


In [4]:
# 🔧 ETAPA: GridSearch Manual com ParameterGrid, tqdm e Tracking MLflow

"""
Executa:
1) Normaliza o CWD para '/workspace'.
2) Carrega train_curated e test_curated.
3) Reconstrói y a partir das colunas dummy.
4) Itera ParameterGrid manualmente.
5) Barra tqdm avança por combinação.
6) Loga cada combinação como run separado no MLflow.
7) Prints coerentes com links 127.0.0.1.
"""

import os
import logging
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm

# ✅ Silenciar logger redundante do MLflow
logging.getLogger("mlflow").setLevel(logging.ERROR)

# 1️⃣ Normalizar CWD
print("CWD antes:", os.getcwd())
os.chdir('/workspace')
print("CWD depois:", os.getcwd())

# 2️⃣ Paths
TRAIN_PATH = 'data/curated/train_curated.csv'
TEST_PATH = 'data/curated/test_curated.csv'

# 3️⃣ Carga + reconstrução y
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

dummy_cols = [col for col in train_df.columns if col.startswith('Credit_Score_')]
print("Colunas classe:", dummy_cols)

y_train = train_df[dummy_cols].idxmax(axis=1).str.replace('Credit_Score_', '')
X_train = train_df.drop(columns=dummy_cols)

y_test = test_df[dummy_cols].idxmax(axis=1).str.replace('Credit_Score_', '')
X_test = test_df.drop(columns=dummy_cols)

print(f"X_train: {X_train.shape} | y_train: {y_train.shape}")

# 4️⃣ ParameterGrid manual
param_grid = {
    "max_depth": [3, 5, 7],
    "min_samples_split": [2, 5, 10]
}
grid = ParameterGrid(param_grid)

# 5️⃣ Tracking URI + credenciais MinIO
mlflow.set_tracking_uri("http://mlflow:5000")
os.environ['AWS_ACCESS_KEY_ID'] = 'wrm'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'senha_segura'
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://minio:9000'

experiment_name = "QuantumFinance_CreditScore"
mlflow.set_experiment(experiment_name)

print("\nExecutando GridSearch manual...")

for params in tqdm(grid, desc="Runs"):
    with mlflow.start_run(run_name=f"GridSearch_Manual_{params}") as run:
        model = DecisionTreeClassifier(**params, random_state=42)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')

        mlflow.log_params(params)
        mlflow.log_metric("accuracy", acc)
        mlflow.log_metric("f1_score", f1)
        mlflow.sklearn.log_model(model, "model")

        print(f"\nCombinação: {params}")
        print(f"Accuracy: {acc:.4f} | F1 Score: {f1:.4f}")
        print(f"Run ID: {run.info.run_id}")
        print(f"Acesse: http://127.0.0.1:5000/#/experiments/{run.info.experiment_id}/runs/{run.info.run_id}")


CWD antes: /workspace
CWD depois: /workspace
Colunas classe: ['Credit_Score_Poor', 'Credit_Score_Standard']
X_train: (100000, 6303) | y_train: (100000,)

Executando GridSearch manual...


Runs:  11%|█         | 1/9 [00:15<02:06, 15.75s/it]


Combinação: {'max_depth': 3, 'min_samples_split': 2}
Accuracy: 0.5388 | F1 Score: 0.7003
Run ID: 8347a501625844da89aaa891510faa55
Acesse: http://127.0.0.1:5000/#/experiments/1/runs/8347a501625844da89aaa891510faa55
🏃 View run GridSearch_Manual_{'max_depth': 3, 'min_samples_split': 2} at: http://mlflow:5000/#/experiments/1/runs/8347a501625844da89aaa891510faa55
🧪 View experiment at: http://mlflow:5000/#/experiments/1


Runs:  22%|██▏       | 2/9 [00:27<01:34, 13.56s/it]


Combinação: {'max_depth': 3, 'min_samples_split': 5}
Accuracy: 0.5388 | F1 Score: 0.7003
Run ID: 9f9ded00e4594c7197535ae840bc8bfc
Acesse: http://127.0.0.1:5000/#/experiments/1/runs/9f9ded00e4594c7197535ae840bc8bfc
🏃 View run GridSearch_Manual_{'max_depth': 3, 'min_samples_split': 5} at: http://mlflow:5000/#/experiments/1/runs/9f9ded00e4594c7197535ae840bc8bfc
🧪 View experiment at: http://mlflow:5000/#/experiments/1


...


Combinação: {'max_depth': 7, 'min_samples_split': 2}
Accuracy: 0.5422 | F1 Score: 0.7032
Run ID: 9c9d83b3de294b5aab2357b42a67aa06
Acesse: http://127.0.0.1:5000/#/experiments/1/runs/9c9d83b3de294b5aab2357b42a67aa06
🏃 View run GridSearch_Manual_{'max_depth': 7, 'min_samples_split': 2} at: http://mlflow:5000/#/experiments/1/runs/9c9d83b3de294b5aab2357b42a67aa06
🧪 View experiment at: http://mlflow:5000/#/experiments/1


Runs:  89%|████████▉ | 8/9 [01:35<00:11, 11.69s/it]


Combinação: {'max_depth': 7, 'min_samples_split': 5}
Accuracy: 0.5422 | F1 Score: 0.7032
Run ID: d2685d32bdcb42b783e358095e9f5b63
Acesse: http://127.0.0.1:5000/#/experiments/1/runs/d2685d32bdcb42b783e358095e9f5b63
🏃 View run GridSearch_Manual_{'max_depth': 7, 'min_samples_split': 5} at: http://mlflow:5000/#/experiments/1/runs/d2685d32bdcb42b783e358095e9f5b63
🧪 View experiment at: http://mlflow:5000/#/experiments/1


Runs: 100%|██████████| 9/9 [01:46<00:00, 11.88s/it]


Combinação: {'max_depth': 7, 'min_samples_split': 10}
Accuracy: 0.5424 | F1 Score: 0.7033
Run ID: 0cfc26f47de549969c8fdb2ceef4565c
Acesse: http://127.0.0.1:5000/#/experiments/1/runs/0cfc26f47de549969c8fdb2ceef4565c
🏃 View run GridSearch_Manual_{'max_depth': 7, 'min_samples_split': 10} at: http://mlflow:5000/#/experiments/1/runs/0cfc26f47de549969c8fdb2ceef4565c
🧪 View experiment at: http://mlflow:5000/#/experiments/1


In [5]:
# 🔧 ETAPA: GridSearch Manual com RandomForest, ParameterGrid, tqdm e Tracking MLflow

"""
Executa:
1) Normaliza o CWD para '/workspace'.
2) Carrega train_curated e test_curated.
3) Reconstrói y a partir das colunas dummy.
4) Executa GridSearch manual com RandomForestClassifier.
5) Usa ParameterGrid + tqdm para barra real por combinação.
6) Loga cada run no MLflow com credenciais MinIO coerentes.
7) Prints finais coerentes com links 127.0.0.1.
"""

import os
import logging
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm

# ✅ Silenciar logger redundante do MLflow
logging.getLogger("mlflow").setLevel(logging.ERROR)

# 1️⃣ Normalizar CWD
print("CWD antes:", os.getcwd())
os.chdir('/workspace')
print("CWD depois:", os.getcwd())

# 2️⃣ Paths
TRAIN_PATH = 'data/curated/train_curated.csv'
TEST_PATH = 'data/curated/test_curated.csv'

# 3️⃣ Carga + reconstrução y
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

dummy_cols = [col for col in train_df.columns if col.startswith('Credit_Score_')]
print("Colunas classe:", dummy_cols)

y_train = train_df[dummy_cols].idxmax(axis=1).str.replace('Credit_Score_', '')
X_train = train_df.drop(columns=dummy_cols)

y_test = test_df[dummy_cols].idxmax(axis=1).str.replace('Credit_Score_', '')
X_test = test_df.drop(columns=dummy_cols)

print(f"X_train: {X_train.shape} | y_train: {y_train.shape}")

# 4️⃣ ParameterGrid com RandomForest
param_grid = {
    "n_estimators": [50, 100],
    "max_depth": [3, 5],
    "min_samples_split": [2, 5]
}
grid = ParameterGrid(param_grid)

# 5️⃣ Tracking URI + credenciais MinIO
mlflow.set_tracking_uri("http://mlflow:5000")
os.environ['AWS_ACCESS_KEY_ID'] = 'wrm'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'senha_segura'
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://minio:9000'

experiment_name = "QuantumFinance_CreditScore"
mlflow.set_experiment(experiment_name)

print("\nExecutando GridSearch manual RandomForest...")

for params in tqdm(grid, desc="Runs RandomForest"):
    with mlflow.start_run(run_name=f"GridSearch_RF_{params}") as run:
        model = RandomForestClassifier(**params, random_state=42, n_jobs=-1)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')

        mlflow.log_params(params)
        mlflow.log_metric("accuracy", acc)
        mlflow.log_metric("f1_score", f1)
        mlflow.sklearn.log_model(model, "model")

        print(f"\nCombinação: {params}")
        print(f"Accuracy: {acc:.4f} | F1 Score: {f1:.4f}")
        print(f"Run ID: {run.info.run_id}")
        print(f"Acesse: http://127.0.0.1:5000/#/experiments/{run.info.experiment_id}/runs/{run.info.run_id}")


CWD antes: /workspace
CWD depois: /workspace
Colunas classe: ['Credit_Score_Poor', 'Credit_Score_Standard']
X_train: (100000, 6303) | y_train: (100000,)

Executando GridSearch manual RandomForest...


Runs RandomForest:  12%|█▎        | 1/8 [00:10<01:14, 10.66s/it]


Combinação: {'max_depth': 3, 'min_samples_split': 2, 'n_estimators': 50}
Accuracy: 0.1160 | F1 Score: 0.2078
Run ID: 7e1e8f44f2a14b469b513b183d4a0ae5
Acesse: http://127.0.0.1:5000/#/experiments/1/runs/7e1e8f44f2a14b469b513b183d4a0ae5
🏃 View run GridSearch_RF_{'max_depth': 3, 'min_samples_split': 2, 'n_estimators': 50} at: http://mlflow:5000/#/experiments/1/runs/7e1e8f44f2a14b469b513b183d4a0ae5
🧪 View experiment at: http://mlflow:5000/#/experiments/1


Runs RandomForest:  25%|██▌       | 2/8 [00:18<00:55,  9.26s/it]


Combinação: {'max_depth': 3, 'min_samples_split': 2, 'n_estimators': 100}
Accuracy: 0.0560 | F1 Score: 0.1061
Run ID: 7e2a7fd6532a415790f7e0297eaef518
Acesse: http://127.0.0.1:5000/#/experiments/1/runs/7e2a7fd6532a415790f7e0297eaef518
🏃 View run GridSearch_RF_{'max_depth': 3, 'min_samples_split': 2, 'n_estimators': 100} at: http://mlflow:5000/#/experiments/1/runs/7e2a7fd6532a415790f7e0297eaef518
🧪 View experiment at: http://mlflow:5000/#/experiments/1


...


Combinação: {'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 100}
Accuracy: 0.1582 | F1 Score: 0.2732
Run ID: 3a35386b08854259abb939bffb29abb8
Acesse: http://127.0.0.1:5000/#/experiments/1/runs/3a35386b08854259abb939bffb29abb8
🏃 View run GridSearch_RF_{'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 100} at: http://mlflow:5000/#/experiments/1/runs/3a35386b08854259abb939bffb29abb8
🧪 View experiment at: http://mlflow:5000/#/experiments/1


Runs RandomForest:  88%|████████▊ | 7/8 [00:54<00:07,  7.32s/it]


Combinação: {'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 50}
Accuracy: 0.2117 | F1 Score: 0.3494
Run ID: ee503b2c1af948db890c83f7a178a5de
Acesse: http://127.0.0.1:5000/#/experiments/1/runs/ee503b2c1af948db890c83f7a178a5de
🏃 View run GridSearch_RF_{'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 50} at: http://mlflow:5000/#/experiments/1/runs/ee503b2c1af948db890c83f7a178a5de
🧪 View experiment at: http://mlflow:5000/#/experiments/1


Runs RandomForest: 100%|██████████| 8/8 [01:02<00:00,  7.78s/it]


Combinação: {'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 100}
Accuracy: 0.1581 | F1 Score: 0.2730
Run ID: 4693946ae8bf47a799823ab6e1844b93
Acesse: http://127.0.0.1:5000/#/experiments/1/runs/4693946ae8bf47a799823ab6e1844b93
🏃 View run GridSearch_RF_{'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 100} at: http://mlflow:5000/#/experiments/1/runs/4693946ae8bf47a799823ab6e1844b93
🧪 View experiment at: http://mlflow:5000/#/experiments/1


In [1]:
# 🔧 ETAPA: GridSearch Manual com GradientBoostingClassifier, ParameterGrid, tqdm e MLflow

"""
Executa:
1) Normaliza o CWD para '/workspace'.
2) Carrega 'train_clean.csv' da camada processed.
3) Separa X e y com target original.
4) Aplica pd.get_dummies() no X.
5) Usa train_test_split com stratify.
6) Executa GridSearch manual com GradientBoostingClassifier.
7) Barra tqdm para progresso real.
8) Loga cada run separadamente no MLflow.
"""

import os
import logging
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm

# ✅ Silenciar logger redundante
logging.getLogger("mlflow").setLevel(logging.ERROR)

# 1️⃣ Normalizar CWD
print("CWD antes:", os.getcwd())
os.chdir('/workspace')
print("CWD depois:", os.getcwd())

# 2️⃣ Carregar dados
df = pd.read_csv('data/processed/train_clean.csv')
print("\ndf shape:", df.shape)
print("\ndf.head(5):\n", df.head(5))

# 3️⃣ Separa X e y
X = df.drop(columns=['Credit_Score'])
y = df['Credit_Score']

# 4️⃣ Pré-processa X igual curated
X = pd.get_dummies(X)

# 5️⃣ train_test_split + alinhamento coerente
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

print(f"\nX_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")

# 6️⃣ Configurar ParameterGrid
param_grid = {
    "n_estimators": [50, 100],
    "max_depth": [3, 5],
    "learning_rate": [0.05, 0.1]
}
grid = ParameterGrid(param_grid)

# 7️⃣ Tracking MLflow + MinIO
mlflow.set_tracking_uri("http://mlflow:5000")
os.environ['AWS_ACCESS_KEY_ID'] = 'wrm'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'senha_segura'
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://minio:9000'

experiment_name = "QuantumFinance_CreditScore"
mlflow.set_experiment(experiment_name)

print("\nExecutando GridSearch manual — GradientBoostingClassifier...")

for params in tqdm(grid, desc="Runs GBoost"):
    with mlflow.start_run(run_name=f"GridSearch_GBoost_{params}") as run:
        model = GradientBoostingClassifier(**params, random_state=42)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')

        mlflow.log_params(params)
        mlflow.log_metric("accuracy", acc)
        mlflow.log_metric("f1_score", f1)
        mlflow.sklearn.log_model(model, "model")

        print(f"\nCombinação: {params}")
        print(f"Accuracy: {acc:.4f} | F1 Score: {f1:.4f}")
        print(f"Run ID: {run.info.run_id}")
        print(f"Acesse: http://127.0.0.1:5000/#/experiments/{run.info.experiment_id}/runs/{run.info.run_id}")


CWD antes: /workspace/notebooks
CWD depois: /workspace

df shape: (100000, 28)

df.head(5):
        ID Customer_ID     Month           Name    Age          SSN Occupation  \
0  0x1602   CUS_0xd40   January  Aaron Maashoh   23.0  821-00-0265  Scientist   
1  0x1603   CUS_0xd40  February  Aaron Maashoh   23.0  821-00-0265  Scientist   
2  0x1604   CUS_0xd40     March  Aaron Maashoh -500.0  821-00-0265  Scientist   
3  0x1605   CUS_0xd40     April  Aaron Maashoh   23.0  821-00-0265  Scientist   
4  0x1606   CUS_0xd40       May  Aaron Maashoh   23.0  821-00-0265  Scientist   

   Annual_Income  Monthly_Inhand_Salary  Num_Bank_Accounts  ...  Credit_Mix  \
0       19114.12            1824.843333                  3  ...     Unknown   
1       19114.12                    NaN                  3  ...        Good   
2       19114.12                    NaN                  3  ...        Good   
3       19114.12                    NaN                  3  ...        Good   
4       19114.12         

: 

#  Diagnóstico do Footprint de Memória

Antes de rodar fitting, esta célula calcula o uso real de memória dos DataFrames `X_train` e `y_train`.  
Garante rastreabilidade sobre quanta RAM o kernel irá consumir, considerando `deep=True` para contar objetos, ponteiros e índices.  
Este valor deve ser menor que 70% da RAM real do container para evitar travamento por OOM Killer.


## Recarga dos Datasets Curated — Caminho corrigido

Esta célula recarrega os datasets `train_curated.csv` e `test_curated.csv` usando caminho relativo correto, garantindo coerência com a estrutura `/workspace/`.



In [3]:
# ETAPA: Recarga dos Datasets Curated

import pandas as pd

train_df = pd.read_csv("../data/curated/train_curated.csv")
test_df = pd.read_csv("../data/curated/test_curated.csv")

print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")


Train shape: (100000, 6305)
Test shape: (50000, 6305)


In [7]:
print(train_df.columns.tolist())


['Age', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'Delay_from_due_date', 'Num_Credit_Inquiries', 'Outstanding_Debt', 'Credit_Utilization_Ratio', 'Total_EMI_per_month', 'Amount_invested_monthly', 'Monthly_Balance', 'Num_of_Loan_Bin', 'Changed_Credit_Limit_Bin', 'Num_of_Delayed_Payment_Bin', 'Credit_History_Age_Bin', 'Month_Num', 'Occupation_Architect', 'Occupation_Developer', 'Occupation_Doctor', 'Occupation_Engineer', 'Occupation_Entrepreneur', 'Occupation_Journalist', 'Occupation_Lawyer', 'Occupation_Manager', 'Occupation_Mechanic', 'Occupation_Media_Manager', 'Occupation_Musician', 'Occupation_Scientist', 'Occupation_Teacher', 'Occupation_Unknown', 'Occupation_Writer', 'Type_of_Loan_Auto Loan, Auto Loan, Auto Loan, Auto Loan, Credit-Builder Loan, Credit-Builder Loan, Mortgage Loan, and Personal Loan', 'Type_of_Loan_Auto Loan, Auto Loan, Auto Loan, Auto Loan, Student Loan, and Student Loan', 'Type_of_Loan_Auto Loan, Auto Loan, A

---
REABRIR O FEATURE_ENGINEERING_CURADORIA.IPNYB PARA DIMINUIR CARDINALIDADE
---

---


## Extensão Controlada — One-Hot Encoding Restrito Pré-Cardinalidade

Este bloco aplica o **One-Hot Encoding restrito** nas variáveis `Month`, `Occupation_Group` e `Payment_Behaviour`, antes de qualquer novo diagnóstico de cardinalidade.

O objetivo é observar quantas colunas serão adicionadas e comparar com o pipeline anterior (+6.300 colunas) para verificar se o footprint segue controlado.
O resultado será salvo como `CURATED V1.1` para rastreabilidade total.


In [5]:
# ETAPA: ONE-HOT ENCODING RESTRITO V1.1 E COMPARACAO IMEDIATA

import pandas as pd

# Caminhos
train_curated_v1 = '/workspace/data/curated/train_curated_v1.csv'
test_curated_v1  = '/workspace/data/curated/test_curated_v1.csv'

# Carrega
train_df = pd.read_csv(train_curated_v1)
test_df  = pd.read_csv(test_curated_v1)

# Colunas a codificar
cols_to_encode = ['Month', 'Occupation_Group', 'Payment_Behaviour']

# Aplica OHE restrito
train_encoded = pd.get_dummies(train_df, columns=cols_to_encode, drop_first=True)
test_encoded  = pd.get_dummies(test_df, columns=cols_to_encode, drop_first=True)

# Alinha colunas para garantir mesma estrutura
train_encoded, test_encoded = train_encoded.align(test_encoded, join='outer', axis=1, fill_value=0)

# Compara shape
print("\nShape original V1 (train):", train_df.shape)
print("Shape V1.1 após OHE (train):", train_encoded.shape)

print("\nShape original V1 (test):", test_df.shape)
print("Shape V1.1 após OHE (test):", test_encoded.shape)

# Salva V1.1
train_encoded.to_csv('/workspace/data/curated/train_curated_v1_1.csv', index=False)
test_encoded.to_csv('/workspace/data/curated/test_curated_v1_1.csv', index=False)

print("\nSnapshots CURATED V1.1 salvos.")



Shape original V1 (train): (100000, 65)
Shape V1.1 após OHE (train): (100000, 93)

Shape original V1 (test): (50000, 64)
Shape V1.1 após OHE (test): (50000, 93)

Snapshots CURATED V1.1 salvos.


## Versionamento Atômico — Snapshot CURATED V1.1

Este bloco faz o versionamento atômico do `train_curated_v1_1.csv` e `test_curated_v1_1.csv` com `DVC` e `Git`.  
O fluxo garante rastreabilidade total: verificação física, commit coerente, push para backend MinIO.


In [6]:
# ETAPA: VERSIONAMENTO ATÔMICO CURATED V1.1

import os
import subprocess

# Caminhos V1.1
train_curated_v1_1 = '/workspace/data/curated/train_curated_v1_1.csv'
test_curated_v1_1  = '/workspace/data/curated/test_curated_v1_1.csv'

# Verifica CWD
print("\nDiretório de trabalho atual:", os.getcwd())

# Confirma existência física
print("\nVerificando existência física:")
print("TRAIN V1.1:", os.path.exists(train_curated_v1_1))
print("TEST V1.1 :", os.path.exists(test_curated_v1_1))

if not os.path.exists(train_curated_v1_1) or not os.path.exists(test_curated_v1_1):
    raise FileNotFoundError("Um dos arquivos CURATED V1.1 não foi encontrado.")

# DVC add
print("\nExecutando dvc add ...")
subprocess.run(['dvc', 'add', train_curated_v1_1], check=True)
subprocess.run(['dvc', 'add', test_curated_v1_1], check=True)

# Git add dos metadados .dvc
print("\nAdicionando metadados .dvc ao Git ...")
subprocess.run(['git', 'add', f"{train_curated_v1_1}.dvc"], check=True)
subprocess.run(['git', 'add', f"{test_curated_v1_1}.dvc"], check=True)

# Commit coerente
print("\nRealizando commit Git ...")
subprocess.run(['git', 'commit', '-m', 'Versionamento CURATED V1.1 com OHE restrito'], check=True)

# DVC push
print("\nExecutando dvc push ...")
subprocess.run(['dvc', 'push'], check=True)

# Git push final
print("\nExecutando git push ...")
subprocess.run(['git', 'push'], check=True)

print("\nVersionamento CURATED V1.1 concluído com sucesso.")



Diretório de trabalho atual: /workspace/notebooks

Verificando existência física:
TRAIN V1.1: True
TEST V1.1 : True

Executando dvc add ...


⠋ Checking graph
⠋ Checking graph



Adicionando metadados .dvc ao Git ...

Realizando commit Git ...
[main 7d92073] Versionamento CURATED V1.1 com OHE restrito
 2 files changed, 10 insertions(+)
 create mode 100644 data/curated/test_curated_v1_1.csv.dvc
 create mode 100644 data/curated/train_curated_v1_1.csv.dvc

Executando dvc push ...
2 files pushed

Executando git push ...

Versionamento CURATED V1.1 concluído com sucesso.


To github.com:WRMELO/MBA_MLOPS.git
   f76be84..7d92073  main -> main


## Diagnóstico de Footprint — Snapshot CURATED V1.1

Este bloco confirma o footprint em **disco** e **RAM** dos arquivos `CURATED V1.1` para manter padrão comparável ao `V1` e ao pipeline original.

Os valores serão exibidos em MB.


In [7]:
# ETAPA: DIAGNOSTICO FOOTPRINT CURATED V1.1

import os
import pandas as pd

# Funções
def file_size(path):
    size_bytes = os.path.getsize(path)
    return round(size_bytes / (1024 * 1024), 2)

def memory_usage_df(path):
    df = pd.read_csv(path)
    return round(df.memory_usage(deep=True).sum() / (1024 * 1024), 2)

# Caminhos V1.1
train_v1_1 = '/workspace/data/curated/train_curated_v1_1.csv'
test_v1_1  = '/workspace/data/curated/test_curated_v1_1.csv'

# Disco
train_disk = file_size(train_v1_1)
test_disk  = file_size(test_v1_1)

# RAM
train_mem = memory_usage_df(train_v1_1)
test_mem  = memory_usage_df(test_v1_1)

print("\nTamanho em DISCO (MB):")
print(f"TRAIN V1.1: {train_disk} MB")
print(f"TEST V1.1 : {test_disk} MB")

print("\nFootprint em MEMÓRIA (MB):")
print(f"TRAIN V1.1: {train_mem} MB")
print(f"TEST V1.1 : {test_mem} MB")



Tamanho em DISCO (MB):
TRAIN V1.1: 63.33 MB
TEST V1.1 : 30.67 MB

Footprint em MEMÓRIA (MB):
TRAIN V1.1: 165.26 MB
TEST V1.1 : 81.38 MB


In [8]:
# ETAPA: RECARREGAMENTO E COMPARACAO SHAPE FINAL CURATED V1.1

import pandas as pd

# Caminhos coerentes V1.1
train_curated_v1_1 = '/workspace/data/curated/train_curated_v1_1.csv'
test_curated_v1_1  = '/workspace/data/curated/test_curated_v1_1.csv'

# Recarrega DataFrames
train_df_v1_1 = pd.read_csv(train_curated_v1_1)
test_df_v1_1  = pd.read_csv(test_curated_v1_1)

# Exibe shapes
print("\nShape atual do TRAIN CURATED V1.1:", train_df_v1_1.shape)
print("Shape atual do TEST CURATED V1.1 :", test_df_v1_1.shape)



Shape atual do TRAIN CURATED V1.1: (100000, 93)
Shape atual do TEST CURATED V1.1 : (50000, 93)


## Comparativo de Footprint — Antes, CURATED V1 e CURATED V1.1

Antes da aplicação do binning supervisionado e agrupamentos controlados, o pipeline de Feature Engineering gerava um conjunto com **altíssima cardinalidade**, alcançando **6.305 colunas** por amostra.

- **Train shape (antigo)**: 100.000 linhas × 6.305 colunas  
- **Test shape (antigo)**: 50.000 linhas × 6.305 colunas

Esse volume extremo era causado por **one-hot indiscriminado** em categorias raras e variáveis contínuas pulverizadas, levando a estouros de memória (OOM Killer) mesmo em máquinas robustas.

Após a revisão completa, com:
- Diagnóstico estatístico detalhado,
- Binning supervisionado com faixas coerentes ao negócio,
- Agrupamento de categorias raras,
- Eliminação de redundâncias mantendo rastreabilidade,

o footprint caiu drasticamente para:

- **Train shape (CURATED V1)**: 100.000 linhas × 65 colunas  
- **Test shape (CURATED V1)**: 50.000 linhas × 64 colunas

Para garantir interpretabilidade e previsão de sazonalidade e perfis de comportamento, foi aplicada uma extensão com **One-Hot Encoding restrito** apenas em variáveis estratégicas (`Month`, `Occupation_Group`, `Payment_Behaviour`):

- **Train shape (CURATED V1.1)**: 100.000 linhas × 93 colunas  
- **Test shape (CURATED V1.1)**: 50.000 linhas × 93 colunas

Assim, a dimensionalidade total foi reduzida de mais de 6.300 colunas para **apenas 93**, viabilizando fitting local, interpretabilidade real e rastreabilidade completa conforme o **PROTOCOLO V5.4**.


In [11]:
# ETAPA: DIAGNOSTICO FINAL DE STRINGS RESIDUAIS EM X

import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Carrega V1.1
df = pd.read_csv('/workspace/data/curated/train_curated_v1_1.csv')

# Separa X e y
target = 'Credit_Score'
X = df.drop(columns=[target])
y = df[target]

# Verifica tipos
print("\nDtypes em X antes do encoding:")
print(X.dtypes.value_counts())

# Identifica colunas object
object_cols = X.select_dtypes(include=['object']).columns.tolist()
print("\nColunas object detectadas:", object_cols)

# Exibe valores únicos por coluna para auditoria
for col in object_cols:
    print(f"\nValores únicos em {col}:", X[col].unique())

# Aplica LabelEncoder
for col in object_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))

# Verifica resultado
print("\nDtypes em X depois do encoding:")
print(X.dtypes.value_counts())

# Split coerente depois de corrigir tudo
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)



Dtypes em X antes do encoding:
bool       50
object     22
float64    13
int64       7
Name: count, dtype: int64

Colunas object detectadas: ['Age_Binned', 'Amount_invested_monthly_Binned', 'Annual_Income_Binned', 'Changed_Credit_Limit_Binned', 'Credit_History_Age', 'Credit_History_Age_Binned', 'Credit_Mix', 'Credit_Utilization_Ratio_Binned', 'Delay_from_due_date_Binned', 'Interest_Rate_Binned', 'Monthly_Balance_Binned', 'Monthly_Inhand_Salary_Binned', 'Num_Bank_Accounts_Binned', 'Num_Credit_Card_Binned', 'Num_Credit_Inquiries_Binned', 'Num_of_Delayed_Payment_Binned', 'Num_of_Loan_Binned', 'Occupation', 'Outstanding_Debt_Binned', 'Payment_of_Min_Amount', 'Total_EMI_per_month_Binned', 'Type_of_Loan']

Valores únicos em Age_Binned: ['Jovem' 'Adulto' 'Idoso' 'Erro']

Valores únicos em Amount_invested_monthly_Binned: ['Baixo' 'Nenhum' 'Moderado' 'Alto']

Valores únicos em Annual_Income_Binned: ['Baixa' 'Média' 'Alta' 'Muito_Alta']

Valores únicos em Changed_Credit_Limit_Binned: ['Aumento_

---
## Baseline Supervisionado — CURATED V1 com Tracking MLflow

Este bloco executa o **fitting baseline** usando a camada `CURATED V1` otimizada.  
A execução usa Árvore de Decisão com profundidade controlada, registrando métricas principais no **MLflow**, garantindo rastreabilidade integral do experimento.

Configuração:
- Target: `Credit_Score`
- Features: Todas as colunas numéricas, binned e agrupadas, exceto ID e texto redundante
- Tracking URI: interno (`http://mlflow:5000`) coerente com container


---
## Fitting Baseline — Snapshot CURATED V1.1 com Pré-processamento Correto

Este bloco executa o **fitting supervisionado baseline** com o `CURATED V1.1`,  
usando o `X_train` limpo com todas as colunas `_Binned` e agrupamentos **convertidos para valores numéricos** via `LabelEncoder`.

A execução usa Árvore de Decisão (`max_depth=5`), com rastreamento no **mesmo projeto MLflow**, mantendo coerência de URI local e acesso em `http://127.0.0.1:5000`.


In [13]:
import os

# 🔐 Variáveis persistentes para o boto3
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://127.0.0.1:9000'
os.environ['AWS_ACCESS_KEY_ID'] = 'wrm'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'senha_segura'

print("✅ Credenciais MinIO configuradas para MLflow.")


✅ Credenciais MinIO configuradas para MLflow.


In [15]:
# ETAPA: FITTING BASELINE FINAL — CURATED V1.1 COM ENDPOINT CORRETO

import os
import mlflow
import mlflow.sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score

# Força o endpoint S3 para o nome do serviço dentro da rede Docker
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://minio:9000'
os.environ['AWS_ACCESS_KEY_ID'] = 'wrm'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'senha_segura'

print("✅ Endpoint S3 dentro do container:", os.environ['MLFLOW_S3_ENDPOINT_URL'])

# Tracking MLflow coerente
mlflow.set_tracking_uri("http://mlflow:5000")
mlflow.set_experiment("Baseline_Curated_V1.1")

with mlflow.start_run():
    clf = DecisionTreeClassifier(max_depth=5, random_state=42)
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_val)

    acc = accuracy_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred, average='macro')

    mlflow.log_param("model_type", "DecisionTreeClassifier")
    mlflow.log_param("max_depth", 5)
    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("f1_macro", f1)

    mlflow.sklearn.log_model(clf, "model_baseline_v1_1")

print(f"\nBaseline concluído | Accuracy: {round(acc,4)} | F1 Macro: {round(f1,4)}")
print("Acesse o MLflow Tracking UI fora do container em: http://127.0.0.1:5000")


✅ Endpoint S3 dentro do container: http://minio:9000


2025/07/15 16:44:14 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/15 16:44:15 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run receptive-lynx-60 at: http://mlflow:5000/#/experiments/3/runs/de0cd9753dd74e6b8f7d5c26663add2f
🧪 View experiment at: http://mlflow:5000/#/experiments/3

Baseline concluído | Accuracy: 0.6881 | F1 Macro: 0.6519
Acesse o MLflow Tracking UI fora do container em: http://127.0.0.1:5000


## Grid Search supervisionado — CURATED V1.1 com Tracking MLflow

Este bloco executa o **Grid Search supervisionado** para o `DecisionTreeClassifier` usando o `CURATED V1.1`.  
Será utilizado:
- Mesma base `X_train` e `y_train` já codificados.
- `GridSearchCV` do scikit-learn.
- Tracking de cada combinação de hiperparâmetros no **mesmo experimento MLflow**, garantindo rastreabilidade integral de métricas.

O objetivo é encontrar a combinação ótima de `max_depth` e `min_samples_split` que maximize o **F1 Macro**.


In [20]:
# ETAPA: GRID SEARCH SUPERVISIONADO COM SCORING MULTICLASSE — DECISION TREE

import mlflow
import mlflow.sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold

# 1️⃣ Define o classificador e os hiperparâmetros
clf = DecisionTreeClassifier(random_state=42)
param_grid = {
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 5, 10, 20]
}

# 2️⃣ Usa StratifiedKFold para garantir estratificação das classes
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 3️⃣ Executa o GridSearchCV com f1_macro
grid_search = GridSearchCV(
    estimator=clf,
    param_grid=param_grid,
    cv=cv,
    scoring='f1_macro',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Melhores parâmetros: {best_params}")
print(f"Melhor F1 Macro: {round(best_score, 4) if best_score is not None else 'N/A'}")

# 4️⃣ Loga no MLflow (evita persistir score NaN)
with mlflow.start_run(run_name="grid_search_decision_tree"):
    mlflow.log_params(best_params)
    if best_score is not None and not (best_score != best_score):  # NaN check
        mlflow.log_metric("best_f1_macro", best_score)
    else:
        print("⚠️ Score é nan — métrica não será logada para evitar conflito de chave")

    # Loga o modelo treinado
    mlflow.sklearn.log_model(
        sk_model=grid_search.best_estimator_,
        artifact_path="grid_search_model",
        input_example=X_train.iloc[:5, :]  # Opcional: remove se não quiser warning
    )

print(f"\n🏃 GridSearch concluído | Best F1 Macro: {round(best_score, 4) if best_score is not None else 'N/A'} | Parâmetros: {best_params}")
print("Acesse o MLflow Tracking UI fora do container em: http://127.0.0.1:5000")


Fitting 5 folds for each of 16 candidates, totalling 80 fits


2025/07/15 16:53:26 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Melhores parâmetros: {'max_depth': 10, 'min_samples_split': 10}
Melhor F1 Macro: 0.6828


/usr/local/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


🏃 View run grid_search_decision_tree at: http://mlflow:5000/#/experiments/3/runs/98db11909ed746028a61ba13a6b9609e
🧪 View experiment at: http://mlflow:5000/#/experiments/3

🏃 GridSearch concluído | Best F1 Macro: 0.6828 | Parâmetros: {'max_depth': 10, 'min_samples_split': 10}
Acesse o MLflow Tracking UI fora do container em: http://127.0.0.1:5000


In [27]:
# 🔧 ETAPA: PREPARAÇÃO DOS DADOS E SPLIT

"""
Este bloco recria X e y, realiza train_test_split com estratificação,
e imprime formas e classes para garantir coerência.
"""

from sklearn.model_selection import train_test_split

# 1️⃣ Define X e y (ajuste o nome real se não for 'Credit_Score')
X = df.drop('Credit_Score', axis=1)
y = df['Credit_Score']

# 2️⃣ Split com estratificação
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# 3️⃣ Checa formas e classes
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")
print(f"Classes: {y_train.unique()}")


X_train shape: (70000, 92)
X_test shape: (30000, 92)
y_train shape: (70000,)
y_test shape: (30000,)
Classes: ['Standard' 'Poor' 'Good']


In [29]:
# 🔧 ETAPA: ENCODING + IMPUTAÇÃO + LOGISTIC REGRESSION BASELINE

"""
Bloco autocontido:
1️⃣ Diagnostica tipos de dados
2️⃣ Aplica OneHotEncoder em colunas categóricas
3️⃣ Junta tudo em matriz X final
4️⃣ Imputa NaN com média nas numéricas
5️⃣ Treina Logistic Regression robusto para multiclasses
6️⃣ Loga tudo no MLflow
"""

import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score
import mlflow
import mlflow.sklearn

# 1️⃣ Diagnóstico de tipos
print("Diagnóstico inicial:")
print(X_train.dtypes)

# 2️⃣ Identifica colunas
categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols   = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()

print(f"Categóricas: {categorical_cols}")
print(f"Numéricas: {numerical_cols}")

# 3️⃣ Pipeline de pré-processamento
preprocessor = ColumnTransformer([
    ('num', SimpleImputer(strategy='mean'), numerical_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
])

# 4️⃣ Pipeline final com Logistic Regression
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(
        max_iter=1000, 
        solver='lbfgs',
        multi_class='multinomial'
    ))
])

# 5️⃣ Ajusta pipeline
pipeline.fit(X_train, y_train)

# 6️⃣ Predição e métricas
y_pred = pipeline.predict(X_test)
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')

# 7️⃣ MLflow Tracking
with mlflow.start_run(run_name="logistic_regression_with_encoding_imputation", experiment_id=3):
    mlflow.log_param("solver", "lbfgs")
    mlflow.log_param("multi_class", "multinomial")
    mlflow.log_param("max_iter", 1000)
    mlflow.log_param("imputer_strategy", "mean")
    mlflow.log_param("encoding", "OneHot")
    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("f1_macro", f1)
    mlflow.sklearn.log_model(pipeline, "logistic_regression_pipeline")

print(f"\n✅ Logistic Regression Baseline | Accuracy: {round(acc,4)} | F1 Macro: {round(f1,4)}")
print("Acesse o MLflow Tracking UI fora do container em: http://127.0.0.1:5000")


Diagnóstico inicial:
Age                                    float64
Age_Binned                              object
Amount_invested_monthly                float64
Amount_invested_monthly_Binned          object
Amount_invested_monthly_Binned_High       bool
                                        ...   
Type_of_Loan_Category_Mortgage Loan       bool
Type_of_Loan_Category_Not Specified       bool
Type_of_Loan_Category_Payday Loan         bool
Type_of_Loan_Category_Personal Loan       bool
Type_of_Loan_Category_Student Loan        bool
Length: 92, dtype: object
Categóricas: ['Age_Binned', 'Amount_invested_monthly_Binned', 'Annual_Income_Binned', 'Changed_Credit_Limit_Binned', 'Credit_History_Age', 'Credit_History_Age_Binned', 'Credit_Mix', 'Credit_Utilization_Ratio_Binned', 'Delay_from_due_date_Binned', 'Interest_Rate_Binned', 'Monthly_Balance_Binned', 'Monthly_Inhand_Salary_Binned', 'Num_Bank_Accounts_Binned', 'Num_Credit_Card_Binned', 'Num_Credit_Inquiries_Binned', 'Num_of_Delayed_Paymen

/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1264: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
2025/07/15 17:11:10 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/15 17:11:11 W

🏃 View run logistic_regression_with_encoding_imputation at: http://mlflow:5000/#/experiments/3/runs/7edcbfe85b1440bba3fde06fe3b76615
🧪 View experiment at: http://mlflow:5000/#/experiments/3

✅ Logistic Regression Baseline | Accuracy: 0.5415 | F1 Macro: 0.355
Acesse o MLflow Tracking UI fora do container em: http://127.0.0.1:5000


# Justificativa Técnica — Normalização Pontual para Modelos Sensíveis a Escala

Para preservar rastreabilidade e reuso do dataset **v1.1**, decidimos:
- Manter o **dataset v1.1** **inalterado** (todas as colunas originais, sem modificação física no arquivo ou tabela).
- Aplicar **normalização apenas sobre as variáveis numéricas** **em memória**, usando `StandardScaler` do `sklearn`.
- Esta normalização é **temporária**, feita **na etapa de treino** para os modelos que exigem features na mesma escala (por exemplo: Regressão Logística, SVM, KNN, Redes Neurais).

**Por que não normalizar todo o dataset na origem?**  
Manter o dataset bruto facilita auditoria, debug de features e comparação de pipelines com/sem pré-processamento.

Portanto:
- Dataset **v1.1** = base única e rastreável.
- Normalização = aplicada **em pipeline**, em **X_train/X_test**, apenas nas colunas numéricas.



In [31]:
# 🔧 ETAPA: NORMALIZAÇÃO PADRÃO DAS VARIÁVEIS NUMÉRICAS

"""
Esta célula aplica StandardScaler somente nas variáveis numéricas do dataset v1.1.
"""

from sklearn.preprocessing import StandardScaler

# Exemplo: defina explicitamente suas numéricas confirmadas
numerical_features = [
    'Age', 'Amount_invested_monthly', 'Annual_Income', 'Changed_Credit_Limit',
    'Credit_History_Age_Months', 'Credit_Utilization_Ratio', 'Delay_from_due_date',
    'Interest_Rate', 'Monthly_Balance', 'Monthly_Inhand_Salary',
    'Num_Bank_Accounts', 'Num_Credit_Card', 'Num_Credit_Inquiries',
    'Num_of_Delayed_Payment', 'Num_of_Loan', 'Outstanding_Debt',
    'Total_EMI_per_month'
    # Ajuste conforme sua lista validada
]

# Inicializa scaler
scaler = StandardScaler()

# Ajusta no treino e transforma treino/teste
X_train_scaled = X_train.copy()
X_test_scaled  = X_test.copy()

X_train_scaled[numerical_features] = scaler.fit_transform(X_train[numerical_features])
X_test_scaled[numerical_features]  = scaler.transform(X_test[numerical_features])

print("✅ Normalização aplicada em memória | Shapes idênticos ao v1.1")
print(f"X_train_scaled shape: {X_train_scaled.shape}")
print(f"X_test_scaled shape : {X_test_scaled.shape}")


✅ Normalização aplicada em memória | Shapes idênticos ao v1.1
X_train_scaled shape: (70000, 92)
X_test_scaled shape : (30000, 92)


In [38]:
# 🔧 ETAPA: IMPUTAÇÃO + NORMALIZAÇÃO + LOGISTIC REGRESSION NUMÉRICO

"""
Recria X_train_num e X_test_num a partir do dataset original (v1_1),
aplica imputação de valores ausentes (média), normaliza com StandardScaler,
ajusta Logistic Regression multinomial robusta e loga tudo no MLflow.
"""

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
import mlflow
import mlflow.sklearn

# 1️⃣ Seleciona colunas numéricas
cols_num = [
    'Age', 'Amount_invested_monthly', 'Annual_Income', 'Changed_Credit_Limit',
    'Credit_History_Age_Months', 'Credit_Utilization_Ratio', 'Delay_from_due_date',
    'Interest_Rate', 'Month_November', 'Month_October', 'Month_September',
    'Monthly_Balance', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts',
    'Num_Credit_Card', 'Num_Credit_Inquiries', 'Num_of_Delayed_Payment',
    'Num_of_Loan', 'Outstanding_Debt', 'Total_EMI_per_month'
]

X_train_num = X_train[cols_num].copy()
X_test_num  = X_test[cols_num].copy()

# 2️⃣ Imputa NaN com média
imputer = SimpleImputer(strategy='mean')
X_train_num_imputed = imputer.fit_transform(X_train_num)
X_test_num_imputed  = imputer.transform(X_test_num)

# 3️⃣ Normaliza
scaler = StandardScaler()
X_train_scaled_num = scaler.fit_transform(X_train_num_imputed)
X_test_scaled_num  = scaler.transform(X_test_num_imputed)

print(f"✅ X_train_scaled_num shape: {X_train_scaled_num.shape}")
print(f"✅ X_test_scaled_num shape : {X_test_scaled_num.shape}")

# 4️⃣ Ajusta Logistic Regression multinomial robusta
logreg = LogisticRegression(
    max_iter=1000,
    solver='lbfgs',
    multi_class='multinomial'
)
logreg.fit(X_train_scaled_num, y_train)

# 5️⃣ Predição e métricas
y_pred = logreg.predict(X_test_scaled_num)
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')

# 6️⃣ MLflow Tracking
with mlflow.start_run(run_name="logistic_regression_num_scaled", experiment_id=3):
    mlflow.log_param("solver", "lbfgs")
    mlflow.log_param("multi_class", "multinomial")
    mlflow.log_param("max_iter", 1000)
    mlflow.log_param("imputer_strategy", "mean")
    mlflow.log_param("scaler", "StandardScaler")
    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("f1_macro", f1)
    mlflow.sklearn.log_model(logreg, "logistic_regression_model")

print(f"\n✅ Logistic Regression Numéricas | Accuracy: {round(acc,4)} | F1 Macro: {round(f1,4)}")
print("Acesse o MLflow Tracking UI fora do container em: http://127.0.0.1:5000")


✅ X_train_scaled_num shape: (70000, 20)
✅ X_test_scaled_num shape : (30000, 20)


/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1264: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
2025/07/15 19:47:21 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/15 19:47:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run logistic_regression_num_scaled at: http://mlflow:5000/#/experiments/3/runs/78d457acce314979815f94fc80d286de
🧪 View experiment at: http://mlflow:5000/#/experiments/3

✅ Logistic Regression Numéricas | Accuracy: 0.5928 | F1 Macro: 0.4974
Acesse o MLflow Tracking UI fora do container em: http://127.0.0.1:5000


##  Justificativa Técnica — Execução do SVM com Variáveis Numéricas Normalizadas

Este bloco marca a continuidade da etapa de experimentação com modelos que exigem dados em escala uniforme.  
O **Support Vector Machine (SVM)** é um algoritmo sensível à magnitude das features — portanto, a **normalização é obrigatória** para maximizar a separabilidade das classes no hiperplano de decisão.

Estamos utilizando:
- **Dataset versão 1.2**, que contém apenas variáveis **numéricas**, já **imputadas** e **normalizadas** com `StandardScaler`.
- Vetores: `X_train_scaled_num` e `X_test_scaled_num`.

Objetivo:
- Gerar um baseline robusto para o SVM dentro do mesmo fluxo rastreável do **MLflow**, garantindo versionamento, consistência de parâmetros (`kernel`, `C`) e comparação justa com os demais algoritmos que também exigem normalização.

Esta execução respeita o protocolo de **blocos autocontidos**, com cabeçalho técnico claro e logging completo de parâmetros e métricas.

Após o SVM, o pipeline seguirá para **KNN** e **MLP**, mantendo a mesma estrutura para validação.



In [40]:
# 🔧 ETAPA: SVM com Numéricas Normalizadas

"""
Este bloco ajusta o modelo Support Vector Machine (SVM)
usando exclusivamente o vetor numérico imputado e normalizado (v1.2).
Inclui ajuste, predição, métricas e logging no MLflow.
"""

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score
import mlflow
import mlflow.sklearn

# 1️⃣ Instancia o modelo SVM
svm = SVC(kernel='rbf', C=1.0)

# 2️⃣ Ajuste
svm.fit(X_train_scaled_num, y_train)

# 3️⃣ Predição
y_pred = svm.predict(X_test_scaled_num)

# 4️⃣ Métricas
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')

# 5️⃣ MLflow Tracking
with mlflow.start_run(run_name="svm_num_scaled", experiment_id=3):
    mlflow.log_param("kernel", "rbf")
    mlflow.log_param("C", 1.0)
    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("f1_macro", f1)
    mlflow.sklearn.log_model(svm, "svm_model")

print(f"\n✅ SVM Numéricas | Accuracy: {round(acc,4)} | F1 Macro: {round(f1,4)}")
print("Acesse o MLflow Tracking UI fora do container em: http://127.0.0.1:5000")


2025/07/15 19:57:55 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/15 19:57:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run svm_num_scaled at: http://mlflow:5000/#/experiments/3/runs/8148b15e77874ab7ac850fdb0e658e22
🧪 View experiment at: http://mlflow:5000/#/experiments/3

✅ SVM Numéricas | Accuracy: 0.6223 | F1 Macro: 0.4626
Acesse o MLflow Tracking UI fora do container em: http://127.0.0.1:5000


# Registro do Resultado — SVM com Dados Numéricos Normalizados

O SVM foi executado conforme planejado, utilizando o `StandardScaler` para garantir comparabilidade justa e melhor separabilidade do hiperplano.  
- **Dataset:** `v1.2` (numéricas imputadas e normalizadas)  
- **Accuracy:** 0.6223  
- **F1 Macro:** 0.4626  
- **Run MLflow:** [Link do Run](http://mlflow:5000/#/experiments/3/runs/8148b15e77874ab7ac850fdb0e658e22)

Esta etapa reforça a necessidade de manter a padronização para algoritmos sensíveis a escala, além de documentar o versionamento para rastreabilidade total do pipeline.

**Próximos passos:**  
Prosseguir com o **K-Nearest Neighbors (KNN)** e o **MLP Classifier**, utilizando os mesmos vetores `X_train_scaled_num` e `X_test_scaled_num` para consolidar a comparação de modelos sensíveis à normalização.


# 🔧 ETAPA: K-Nearest Neighbors — Baseline Numéricas Normalizadas

Esta etapa executa o KNN como parte do bloco de algoritmos que exigem dados normalizados (`v1.2`).  
O objetivo é avaliar o desempenho do KNN usando as mesmas features numéricas previamente escaladas com `StandardScaler`, garantindo comparabilidade entre modelos.  
Todos os parâmetros, métricas e artefatos são rastreados no MLflow, seguindo o protocolo de versionamento.

- **Dataset:** v1.2 — Numéricas imputadas + normalizadas  
- **Observação:** Sem OneHotEncoding, apenas features contínuas
- **Métricas:** Accuracy e F1 Macro  


In [41]:
# 🔧 ETAPA: KNN Baseline Numéricas Normalizadas

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score
import mlflow
import mlflow.sklearn

# 1️⃣ Define e ajusta o KNN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled_num, y_train)

# 2️⃣ Predição e métricas
y_pred = knn.predict(X_test_scaled_num)
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')

# 3️⃣ MLflow
with mlflow.start_run(run_name="knn_num_scaled", experiment_id=3):
    mlflow.log_param("n_neighbors", 5)
    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("f1_macro", f1)
    mlflow.sklearn.log_model(knn, "knn_model")

print(f"\n✅ KNN Numéricas | Accuracy: {round(acc,4)} | F1 Macro: {round(f1,4)}")
print("Acesse o MLflow Tracking UI fora do container em: http://127.0.0.1:5000")


2025/07/15 19:59:56 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/15 19:59:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run knn_num_scaled at: http://mlflow:5000/#/experiments/3/runs/f532e223f98c449fae1e0ce12b6ad03b
🧪 View experiment at: http://mlflow:5000/#/experiments/3

✅ KNN Numéricas | Accuracy: 0.5822 | F1 Macro: 0.5407
Acesse o MLflow Tracking UI fora do container em: http://127.0.0.1:5000


# 🔧 ETAPA: MLP Classifier — Baseline Numéricas Normalizadas

Esta etapa aplica o **Multi-layer Perceptron (MLP Classifier)** ao conjunto `v1.2`  
— contendo apenas variáveis numéricas, imputadas e normalizadas com `StandardScaler`.  
O objetivo é avaliar o comportamento de um modelo de rede neural simples neste cenário, garantindo rastreabilidade no MLflow.  
Todos os hiperparâmetros, métricas e artefatos serão versionados.

- **Dataset:** v1.2 — Numéricas imputadas + normalizadas  
- **Métricas:** Accuracy e F1 Macro  
- **Observação:** O MLP é particularmente sensível a dados não escalados.


In [42]:
# 🔧 ETAPA: MLP Classifier Baseline Numéricas Normalizadas

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score
import mlflow
import mlflow.sklearn

# 1️⃣ Define e ajusta MLP
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=42)
mlp.fit(X_train_scaled_num, y_train)

# 2️⃣ Predição e métricas
y_pred = mlp.predict(X_test_scaled_num)
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')

# 3️⃣ MLflow
with mlflow.start_run(run_name="mlp_num_scaled", experiment_id=3):
    mlflow.log_param("hidden_layer_sizes", "(100,)")
    mlflow.log_param("max_iter", 300)
    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("f1_macro", f1)
    mlflow.sklearn.log_model(mlp, "mlp_model")

print(f"\n✅ MLP Numéricas | Accuracy: {round(acc,4)} | F1 Macro: {round(f1,4)}")
print("Acesse o MLflow Tracking UI fora do container em: http://127.0.0.1:5000")


2025/07/15 20:01:16 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/15 20:01:17 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run mlp_num_scaled at: http://mlflow:5000/#/experiments/3/runs/aa47219a2fc14174a3fb8851ad6bc814
🧪 View experiment at: http://mlflow:5000/#/experiments/3

✅ MLP Numéricas | Accuracy: 0.6618 | F1 Macro: 0.6086
Acesse o MLflow Tracking UI fora do container em: http://127.0.0.1:5000


# 🔧 ETAPA: Modelos Ensemble com Dataset Original v1.1

Esta etapa retoma o dataset original `v1.1`  
— já com **OneHotEncoding**, imputação apropriada e sem normalização —  
para treinar e avaliar os modelos baseados em árvores e ensemble:

- **Decision Tree** (já executado previamente, servirá de comparação)
- **Random Forest**
- **XGBoost**
- **LightGBM**

Esses algoritmos não exigem dados normalizados, pois suas divisões e pesos são determinados por relações de ordenamento, não por distância.
Cada modelo será logado no MLflow com parâmetros e métricas.


# Diagnóstico das Colunas do Tipo Object

Este bloco realiza um diagnóstico técnico preciso das colunas que ainda estão no tipo `object` dentro de `X_train` e `X_test`.  
A execução desta etapa é obrigatória porque algoritmos baseados em árvore, como **Random Forest**, **XGBoost**, **LightGBM** e **HistGradientBoosting**, **não aceitam variáveis categóricas em formato `object` ou `string`** — eles exigem que todos os dados de entrada estejam em formato **numérico**.

Além disso, é importante garantir que não existam valores ausentes (`NaN`) antes do treinamento, pois mesmo esses algoritmos que toleram alguns `NaN` podem apresentar comportamento instável ou inviabilizar splits corretos na árvore.

Portanto, o procedimento faz três verificações fundamentais:
1. Mapeia todas as colunas `object` em `X_train` para confirmar quais variáveis precisam de transformação via **OrdinalEncoder**.
2. Mostra o número de valores únicos em cada coluna e exemplos de categorias, para detectar cardinalidades incoerentes ou inconsistências.
3. Identifica a quantidade de valores `NaN` em cada coluna `object`, embasando a estratégia de imputação.

Este diagnóstico garante que, na próxima etapa, todo o pipeline de imputação e encoding seja construído com **consistência e rastreabilidade**, mantendo a coerência com a regra principal do projeto: **não usar OneHotEncoder** que infle a dimensionalidade e não violar o limite de colunas definido.


In [44]:
# ETAPA: Diagnóstico de colunas object

# Mapeamento e verificação de colunas do tipo object em X_train e X_test

print("\nResumo de tipos em X_train:")
print(X_train.dtypes.value_counts())

print("\nColunas do tipo object em X_train:")
object_cols_train = X_train.select_dtypes(include='object').columns.tolist()
print(object_cols_train)

for col in object_cols_train:
    print(f"\nColuna: {col}")
    uniques = X_train[col].unique()
    nunique = X_train[col].nunique()
    print(f"Valores únicos ({nunique}): {uniques[:20]}")
    if nunique > 20:
        print(f"... ({nunique - 20} valores adicionais não exibidos)")
    print(f"Qtd NaNs: {X_train[col].isna().sum()}")

print("\nVerificação final:")
print(f"Total de colunas object em X_train: {len(object_cols_train)}")



Resumo de tipos em X_train:
bool       50
object     22
float64    13
int64       7
Name: count, dtype: int64

Colunas do tipo object em X_train:
['Age_Binned', 'Amount_invested_monthly_Binned', 'Annual_Income_Binned', 'Changed_Credit_Limit_Binned', 'Credit_History_Age', 'Credit_History_Age_Binned', 'Credit_Mix', 'Credit_Utilization_Ratio_Binned', 'Delay_from_due_date_Binned', 'Interest_Rate_Binned', 'Monthly_Balance_Binned', 'Monthly_Inhand_Salary_Binned', 'Num_Bank_Accounts_Binned', 'Num_Credit_Card_Binned', 'Num_Credit_Inquiries_Binned', 'Num_of_Delayed_Payment_Binned', 'Num_of_Loan_Binned', 'Occupation', 'Outstanding_Debt_Binned', 'Payment_of_Min_Amount', 'Total_EMI_per_month_Binned', 'Type_of_Loan']

Coluna: Age_Binned
Valores únicos (4): ['Adulto' 'Jovem' 'Idoso' 'Erro']
Qtd NaNs: 0

Coluna: Amount_invested_monthly_Binned
Valores únicos (4): ['Baixo' 'Moderado' 'Alto' 'Nenhum']
Qtd NaNs: 0

Coluna: Annual_Income_Binned
Valores únicos (4): ['Baixa' 'Média' 'Alta' 'Muito_Alta']
Qt